In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import sympy
import scipy.stats as st
import scipy.special
import pandas
import csv
import math
import multiprocessing as mp
import datetime
from decimal import *
from mpl_toolkits.mplot3d import Axes3D
from google.colab import drive

## **Google Drive**

In [5]:
# connect to google drive to save the csv file
drive.mount('drive')
os.chdir("drive/My Drive/Yen-Ting Lin/Code")

Mounted at drive


## **Initial Parameters**

In [82]:
# number of preamble
N_4SRA = [44]
N_2SRA = [10]

# number of max retransmission
retrans = 3 # N_PTmax - 1

# Time Duration (ms)
T_RAREP = 1280
T_RAR = 6.4 + 3
# pp = Rmax * G = 1 * 8
# TS 36.331, ra-ResponseWindowSize-r13 = 2pp
W_RAR = 16
# TS 36.331, max-ContentionResoulutionTimer-r13 = 4pp
T_N_CR = 32
# TS 36.321, Backoff Parameter Values for NB-IoT
W_BO = 4096

def maxslot():
  Maxraslot = np.ceil((T_RAR + W_RAR + W_BO) / T_RAREP) * (retrans + 1) + 1
  return int(Maxraslot)

# total number of UE
M = 1000
# initail number of 2SRA UE
M_4SRA = []
M_2SRA = []
for i in range(1, 1000):
  M_2SRA.append(i)
  M_4SRA.append(M - i)

## **Back-off Calculation**

In [41]:
# alpha with i
def find_kmin(i):
  return np.ceil((i - 1) + ((1 - (T_RAR + W_RAR + W_BO)) / T_RAREP))

def find_kmax(i):
  return np.floor(i - ((T_RAR + W_RAR + 1) / T_RAREP))

def find_alpha(i, k, kmin, kmax):
  if kmin <= k <= (i - ((T_RAR + W_RAR + W_BO) / T_RAREP)):
    return (((k - 1) * T_RAREP) + T_RAR + W_RAR + W_BO - ((i - 2) * T_RAREP)) / W_BO
  elif (i - ((T_RAR + W_RAR + W_BO) / T_RAREP)) < k < ((i - 1) - ((T_RAR + W_RAR) / T_RAREP)):
    return ((T_RAREP) / W_BO)
  elif ((i - 1) - ((T_RAR + W_RAR) / T_RAREP)) <= k <= kmax:
    return (((i - 1) * T_RAREP) - (((k - 1) * T_RAREP) + T_RAR + W_RAR)) / W_BO
  else:
    return 0

## **Main Function**

In [106]:
def main():
  Ms_4SRA_list = [0.0 for i in range(0, len(M_4SRA))]
  Ps_4SRA_list = [0.0 for i in range(0, len(M_4SRA))]
  Da_4SRA_list = [0.0 for i in range(0, len(M_4SRA))]

  Ms_2SRA_list = [0.0 for i in range(0, len(M_2SRA))]
  Ps_2SRA_list = [0.0 for i in range(0, len(M_2SRA))]
  Da_2SRA_list = [0.0 for i in range(0, len(M_2SRA))]

  Ms_T_list = [0.0 for i in range(0, M - 1)]
  Ps_T_list = [0.0 for i in range(0, M - 1)]
  Da_T_list = [0.0 for i in range(0, M - 1)]

  for ue in range(0, M - 1):
    # initial the MiSn and MiFn: Mi[slot][num of trans + 1]
    # the last time of retrans needed to be ignored
    MiCn_4SRA = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    MiSn_4SRA = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    MiFn_4SRA = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    T_4SRA = [0.0 for i in range(0, maxslot() + 1)]

    MiCn_2SRA = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    MiSn_2SRA = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    MiFn_2SRA = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    T_2SRA = [0.0 for i in range(0, maxslot() + 1)]

    # num of total UE in slot 0
    MiFn_4SRA[0][0] = M_4SRA[ue]
    MiFn_2SRA[0][0] = M_2SRA[ue]
    # print(np.matrix(MiFn))

    for i in range(0, maxslot()):
      # print("i = ", i, "\n")
      kmin = find_kmin(i)
      kmax = find_kmax(i)
      # print("kmin = ", kmin, "kmax = ", kmax, "\n")
      m_4SRA = MiFn_4SRA[i][0] + MiFn_4SRA[i][1] + MiFn_4SRA[i][2] + MiFn_4SRA[i][3] # Mi
      m_2SRA = MiFn_2SRA[i][0] + MiFn_2SRA[i][1] + MiFn_2SRA[i][2] + MiFn_2SRA[i][3] # Mi

      # 4SRA procedure
      for n in range(0, int(retrans + 1)):
        # print("n = ", n, "\n")
        k_list = []
        if MiFn_4SRA[i][n] != 0:
          for k in range(int(kmin), int(kmax + 1)):
            # print("k = ", k, "\n")
            if N_4SRA[0] == 0:
              E_M = 0
            else:
              E_M = np.exp(-(m_4SRA / N_4SRA[0]))
            alpha = find_alpha(i, k, kmin, kmax)
            k_list.append(alpha * (MiFn_4SRA[i][n] * (1.0 - E_M)))
            # print("alpha = ", alpha, "\n")
          # process the data to put it into array like type 1
          for itr_k in range(0, len(k_list)):
            MiFn_4SRA[i + itr_k + 1][n + 1] += k_list[(len(k_list) - 1) - itr_k]
            # print("MiCn = ", MiFn[i][n] * (1.0 - E_M), "\n")
            # print(np.matrix(MiFn))
      for n in range(0, int(retrans + 1)):
        if N_4SRA[0] == 0:
          E_M = 0
        else:
          E_M = np.exp(-(m_4SRA / N_4SRA[0]))
        MiSn_4SRA[i][n] = MiFn_4SRA[i][n] * E_M
        MiCn_4SRA[i][n] = MiFn_4SRA[i][n] * (1.0 - E_M)
      # define delay
      T_4SRA[i] = (i + 1) * T_RAREP + T_RAR + W_RAR

      # 2SRA procedure
      for n in range(0, int(retrans + 1)):
        # print("n = ", n, "\n")
        k_list = []
        if MiFn_2SRA[i][n] != 0:
          for k in range(int(kmin), int(kmax + 1)):
            # print("k = ", k, "\n")
            if N_2SRA[0] == 0:
              E_M = 0
            else:
              E_M = np.exp(-(m_2SRA / N_2SRA[0]))
            alpha = find_alpha(i, k, kmin, kmax)
            k_list.append(alpha * (MiFn_2SRA[i][n] * (1.0 - E_M)))
            # print("alpha = ", alpha, "\n")
          # process the data to put it into array like type 1
          for itr_k in range(0, len(k_list)):
            MiFn_2SRA[i + itr_k + 1][n + 1] += k_list[(len(k_list) - 1) - itr_k]
            # print("MiCn = ", MiFn[i][n] * (1.0 - E_M), "\n")
            # print(np.matrix(MiFn))
      for n in range(0, int(retrans + 1)):
        if N_2SRA[0] == 0:
          E_M = 0
        else:
          E_M = np.exp(-(m_2SRA / N_2SRA[0]))
        MiSn_2SRA[i][n] = MiFn_2SRA[i][n] * E_M
        MiCn_2SRA[i][n] = MiFn_2SRA[i][n] * (1.0 - E_M)
      # define delay
      T_2SRA[i] = (i + 1) * T_RAREP + T_RAR + W_RAR

    # Ms, Ps, Da
    for i in range(0, maxslot()):
      for n in range(0, int(retrans + 1)):
        # sum the number of succeed UE for every transmission in each slot
        Ms_4SRA_list[ue] += MiSn_4SRA[i][n]
        Ms_2SRA_list[ue] += MiSn_2SRA[i][n]
        Da_4SRA_list[ue] += MiSn_4SRA[i][n] * float(T_4SRA[i])
        Da_2SRA_list[ue] += MiSn_2SRA[i][n] * float(T_2SRA[i])


    # total number of succeed UE when there have k UE in the system
    Ms_4SRA_list[ue] = round(Ms_4SRA_list[ue] * 1000000) / 1000000
    Ms_2SRA_list[ue] = round(Ms_2SRA_list[ue] * 1000000) / 1000000
    Ms_T_list[ue] = round((Ms_4SRA_list[ue] + Ms_2SRA_list[ue]) * 1000000) / 1000000

    if M_4SRA[ue] != 0:
      Ps_4SRA_list[ue] = round((Ms_4SRA_list[ue] / M_4SRA[ue]) * 1000000) / 1000000
    else:
      Ps_4SRA_list[ue] = 0
    if M_2SRA[ue] != 0:
      Ps_2SRA_list[ue] = round((Ms_2SRA_list[ue] / M_2SRA[ue]) * 1000000) / 1000000
    else:
      Ps_2SRA_list[ue] = 0
    if M_4SRA[ue] + M_2SRA[ue] != 0:
      Ps_T_list[ue] = round((Ms_4SRA_list[ue] + Ms_2SRA_list[ue]) / (M_4SRA[ue] + M_2SRA[ue]) * 1000000) / 1000000
    else:
      Ps_T_list[ue] = 0

    if M_4SRA[ue] != 0:
      Da_4SRA_list[ue] = float(Da_4SRA_list[ue]) / float(Ms_4SRA_list[ue])
    else:
      Da_4SRA_list[ue] = 0.0
    if M_2SRA[ue] != 0:
      Da_2SRA_list[ue] = float(Da_2SRA_list[ue]) / float(Ms_2SRA_list[ue])
    else:
      Da_2SRA_list[ue] = 0.0
    if M_4SRA[ue] + M_2SRA[ue] != 0:
      Da_T_list[ue] = (Da_4SRA_list[ue] * float(Ms_4SRA_list[ue]) + Da_2SRA_list[ue] * float(Ms_2SRA_list[ue])) / (float(Ms_4SRA_list[ue]) + float(Ms_2SRA_list[ue]))
    else:
      Da_T_list[ue] = 0.0

    # print(len(M), len(Ms_list), len(Ps_list), len(Da_list))

  return M_4SRA, Ms_4SRA_list, Ps_4SRA_list, Da_4SRA_list, M_2SRA, Ms_2SRA_list, Ps_2SRA_list, Da_2SRA_list, Ms_T_list, Ps_T_list, Da_T_list

## **Test Result**

In [109]:
columns = ["M_4SRA", "N_4SRA", "Ms_4SRA", "Ps_4SRA", "Da_4SRA", "M_2SRA", "N_2SRA", "Ms_2SRA", "Ps_2SRA", "Da_2SRA", "Ms_T", "Ps_T", "Da_T"]
M_4SRA, Ms_4SRA_list, Ps_4SRA_list, Da_4SRA_list, M_2SRA, Ms_2SRA_list, Ps_2SRA_list, Da_2SRA_list, Ms_T_list, Ps_T_list, Da_T_list = main()

N_4SRA_list = [N_4SRA[0] for i in range(0, M - 1)]
N_2SRA_list = [N_2SRA[0] for i in range(0, M - 1)]

data = {"M_4SRA" : M_4SRA, "N_4SRA" : N_4SRA_list, "Ms_4SRA" : Ms_4SRA_list, "Ps_4SRA" : Ps_4SRA_list, "Da_4SRA" : Da_4SRA_list,
        "M_2SRA" : M_2SRA, "N_2SRA" : N_2SRA_list, "Ms_2SRA" : Ms_2SRA_list, "Ps_2SRA" : Ps_2SRA_list, "Da_2SRA" : Da_2SRA_list,
        "Ms_T" : Ms_T_list, "Ps_T" : Ps_T_list, "Da_T" : Da_T_list}
result = pandas.DataFrame(data)
# cvs_pandas = result.to_csv("ana_testresult.csv", mode='a', index=False, header=True, columns=columns)
cvs_pandas = result.to_csv("ana_testresult.csv", mode='w+', index=False, header=True, columns=columns)

## **Plotting Figures**

In [110]:
ana = pandas.read_csv('ana_testresult.csv')
# sim = pandas.read_csv('sim_testresult.csv')
ana

,M_4SRA,N_4SRA,Ms_4SRA,Ps_4SRA,Da_4SRA,M_2SRA,N_2SRA,Ms_2SRA,Ps_2SRA,Da_2SRA,Ms_T,Ps_T,Da_T
0,999,44,37.602585,0.037640,13299.034524,1,10,1.000000,1.000000,1567.238963,38.602585,0.038603,12995.122349
1,998,44,37.619887,0.037695,13297.001851,2,10,2.000000,1.000000,1808.081984,39.619887,0.039620,12717.044626
2,997,44,37.637232,0.037750,13294.963462,3,10,2.999999,1.000000,2032.891200,40.637231,0.040637,12463.553332
3,996,44,37.654619,0.037806,13292.919752,4,10,3.999995,0.999999,2245.386426,41.654614,0.041655,12232.051007
4,995,44,37.672050,0.037861,13290.870057,5,10,4.999974,0.999995,2448.360138,42.672024,0.042672,12020.429552
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,5,44,5.000000,1.000000,1601.191715,995,10,5.312656,0.005339,14928.679379,10.312656,0.010313,8466.964926
995,4,44,4.000000,1.000000,1544.389178,996,10,5.311200,0.005333,14929.502513,9.311200,0.009311,9179.389386
996,3,44,3.000000,1.000000,1486.481858,997,10,5.309747,0.005326,14930.322313,8.309747,0.008310,10076.802541
997,2,44,2.000000,1.000000,1427.399076,998,10,5.308295,0.005319,14931.144398,7.308295,0.007308,11235.687298
